In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb

#path.join(path.dirname(os.getcwd()), "credentials.ini")

creds_path_ar = ["../../credentials.ini","credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [4]:

    wandb.init(project="3dcnn")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[HR]conv3d'

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: sota-mafat (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200929_102334-25zndzhx
wandb: Syncing run daily-fog-4


In [5]:
cd {PATH_ROOT}

/home/ubuntu/sota-mafat-radar


In [6]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data, get_data_pipeline
from src.models import arch_setup, dataset_ram_reduced, base_base_model, alex_model, base_3d
from src.features import specto_feat

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [7]:
config = dict()
config['num_tracks'] = 3
config['val_ratio'] = 3
config['shift_segment'] = [25]
config['get_shifts'] = True
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False
config['wavelets'] = True

batch_size = 32
lr = 1e-4

In [9]:
full_data = get_data_pipeline.pipeline_trainval_ram_reduced(PATH_DATA, config)

adding auxilary start
adding auxilary end
shift:25


In [10]:
full_data.head()

,segment_id,track_id,geolocation_type,geolocation_id,sensor_id,snr_type,date_index,target_type,doppler_burst,iq_sweep_burst,is_validation,augmentation_info
0,0,11,C,1,1,HighSNR,27,animal,"[14, 14, 14, 43, 43, 43, 15, 15, 15, 17, 14, 1...","[[(6900.381244659424+6699.4177532196045j), (21...",True,[]
1,1,16,C,1,1,HighSNR,27,animal,"[123, 123, 123, 5, 5, 6, 5, 9, 4, 4, 4, 4, 4, ...","[[(-2723.534037590027-36.075279235839844j), (6...",False,[]
2,2,16,C,1,1,HighSNR,27,animal,"[122, 122, 122, 122, 123, 120, 120, 117, 119, ...","[[(2121.38956451416-1027.161826133728j), (159....",False,[]
3,3,29,C,1,1,HighSNR,27,animal,"[75, 75, 75, 55, 32, 31, 33, 36, 30, 31, 31, 2...","[[(1219.9125831127167-2712.076440811157j), (-7...",True,[]
4,4,29,C,1,1,HighSNR,27,animal,"[25, 25, 27, 26, 29, 28, 25, 24, 18, 22, 24, 2...","[[(-1530.8067255020142-2189.1123847961426j), (...",False,[]


In [14]:
train = full_data[full_data.is_validation == False]
validation = full_data[full_data.is_validation == True]




print(train.shape[0])
print(validation.shape[0])

10278
616


In [15]:
from src.models import dataset_ram_reduced
train_set = dataset_ram_reduced.DS(train)
val_set= dataset_ram_reduced.DS(validation)

train_loader=DataLoader(dataset= train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = batch_size, shuffle = True, num_workers = 2)

In [17]:
model= base_3d.alex_3d()
# model.apply(init_weights)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)

if WANDB_enable == True:
    runname = input("Enter WANDB runname(ENTER to skip wandb) :")
    notes = input("Enter run notes :")

    wandb.init(project="3dcnn",name=runname, notes=notes)
    os.environ['WANDB_NOTEBOOK_NAME'] = '[HR]conv3d'
    
    wandb.watch(model)
    wandb.config['data_config'] = config
    wandb.config['train_size'] = train.shape[0]
    wandb.config['val_size'] = validation.shape[0]
    wandb.config['batch_size'] = batch_size
    wandb.config['learning rate'] = lr
    wandb.log(config)


wandb: Waiting for W&B process to finish, PID 28940
wandb: Program ended successfully.

wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200929_103342-16n0jjay/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200929_103342-16n0jjay/logs/debug-internal.log
wandb: Synced 4 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Synced alex3d_take1: https://wandb.ai/sota-mafat/3dcnn/runs/16n0jjay
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200929_103422-27ckc03b
wandb: Syncing run alex3d_1


In [21]:
train_set

In [20]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

TypeError: 'DataLoader' object does not support indexing

In [ ]:
arch_setup.plot_loss_train_test(log,model)

In [ ]:
arch_setup.plot_ROC_local_gpu(train_loader,val_loader,model,device)

In [31]:
importlib.reload(arch_setup)

<module 'src.models.arch_setup' from '/home/ubuntu/sota-mafat-radar/src/models/arch_setup.py'>